In [1]:
import uuid
from stars import BH_stars_img
from telescope_simulator import TelescopeSimulator
import cv2
import pandas as pd

In [2]:
num_stars = 10
num_BHs = 1
num_imgaes = 5000
height = 1024
width = 1024
shape = 'rect'
mode = 'train_val'
# img_lab
# csv

In [3]:
dic = dict(
    # physical parameters
    input_image = r"./stars/BHs.png",
    telescope_diameter_m = 6.5,  
    telescope_focal_length_m = 131.4,
    seeing_arcsec_500nm = .015,
    zenith_angle_deg = 0,
    atmosphere = False,
    wavelength = 50e-9,
    CCD_pixel_size = 6e-5,
    CCD_pixel_count = 1000,
    # simulation parameters
    Num_psfs_to_gen = 10,
    pixels_per_ro = 30,
    show = False,
)

In [4]:
data_dir = f'/mnt/c/fyq/tele_datasets/stars{num_stars}_BH{num_BHs}_num{num_imgaes}_{shape}_wl{dic["wavelength"]:.3e}_D{dic["telescope_diameter_m"]:.2f}_F{dic["telescope_diameter_m"]}'

In [5]:
import json

# Specify the file path
file_path = f"{data_dir}/dictionary.json"

# Save the dictionary to a JSON file
with open(file_path, "w") as json_file:
    json.dump(dic, json_file)


In [6]:
# if mode == 'train_val':
!mkdir {data_dir}
!mkdir {data_dir}/train
!mkdir {data_dir}/train/images
!mkdir {data_dir}/train/labels
!mkdir {data_dir}/validation
!mkdir {data_dir}/validation/images
!mkdir {data_dir}/validation/labels
# elif mode == 'img_lab':
# !mkdir {data_dir}
# !mkdir {data_dir}/images
# !mkdir {data_dir}/images/train
# !mkdir {data_dir}/images/validation
# !mkdir {data_dir}/labels
# !mkdir {data_dir}/labels/train
# !mkdir {data_dir}/labels/validation
# else:
# !mkdir {data_dir}
# !mkdir {data_dir}/images
# !mkdir {data_dir}/labels

In [7]:
df_train = pd.DataFrame(columns=['images', 'labels'])
df_val = pd.DataFrame(columns=['images', 'labels'])

In [8]:
config = dict(
    BHs_path='./224/', num_stars=num_stars, num_BHs=num_BHs, stars_lower_size=7, stars_upper_size=15,
    height=height, width=width, bg_color=0, shape=shape, BHS_lower_size=16, BH_upper_size=20
)

In [9]:
num_train = int(num_imgaes * 0.8)
num_val = min(int(num_imgaes * 0.2), 240)

for i in range(num_train):
    img = BH_stars_img(**config)
    img.stars_gen()
    img.BHs_gen()
    noise_BHs = img.add_noise(img.stars_BHs_img, radius=0)
    dic['input_image'] = noise_BHs
    telescope_simulator = TelescopeSimulator(**dic)
    code = uuid.uuid4()
    if mode == 'train_val':
        bh_path = f"{data_dir}/train/images/BHs_{code}.png"
        txt_path = f"{data_dir}/train/labels/BHs_{code}.txt"
    elif mode == 'img_lab':
        bh_path = f"{data_dir}/images/train/BHs_{code}.png"
        txt_path = f"{data_dir}/labels/train/BHs_{code}.txt"
    else:
        df_train = pd.concat([df_train, pd.DataFrame({"images":[f"BHs_{code}.png"], "labels": [f"BHs_{code}.txt"]})])
        bh_path = f"{data_dir}/images/BHs_{code}.png"
        txt_path = f"{data_dir}/labels/BHs_{code}.txt"
    # print(i)
    # cv2.imwrite(bh_path, noise_BHs)
    telescope_simulator.generate_image(bh_path, show=False)
    img.txtGen(txt_path=txt_path)


In [ ]:
for i in range(num_val):
    img = BH_stars_img(**config)
    img.stars_gen()
    img.BHs_gen()
    noise_BHs = img.add_noise(img.stars_BHs_img, radius=5)
    dic['input_image'] = noise_BHs
    telescope_simulator = TelescopeSimulator(**dic)
    code = uuid.uuid4()
    if mode == 'train_val':
        bh_path = f"{data_dir}/validation/images/BHs_{code}.png"
        txt_path = f"{data_dir}/validation/labels/BHs_{code}.txt"
    elif mode == 'img_lab':
        bh_path = f"{data_dir}/images/validation/BHs_{code}.png"
        txt_path = f"{data_dir}/labels/validation/BHs_{code}.txt"
    else:
        df_val = pd.concat([df_val, pd.DataFrame({"images":[f"BHs_{code}.png"], "labels": [f"BHs_{code}.txt"]})])
        bh_path = f"{data_dir}/images/BHs_{code}.png"
        txt_path = f"{data_dir}/labels/BHs_{code}.txt"
    # cv2.imwrite(bh_path, noise_BHs)
    telescope_simulator.generate_image(bh_path, show=False)
    img.txtGen(txt_path=txt_path)

In [ ]:
if mode == 'csv':
    df_train.to_csv(f"{data_dir}/train.csv", header=False, index=False)
    df_val.to_csv(f"{data_dir}/validation.csv", header=False, index=False)